# Week 4 - Static Attitude Determination

This week, we dive into the fascinating world of static attitude determination. Here, the focus is on taking a snapshot of multiple directional observations—like the sun’s heading, magnetic field direction, and star positioning—to compute a spacecraft's 3D orientation. Essentially, we’re using known vectors in both the spacecraft and reference frames to figure out “which way is up” in space.

We’ll cover both classic and modern algorithms that are fundamental in the field of attitude determination. These include:

- **TRIAD Method**: A foundational, straightforward technique for calculating attitude based on two vector observations.
- **Devenport’s q-Method**: A method that minimizes a cost function to estimate attitude, using quaternions as the rotation representation.
- **QUEST (QUaternion ESTimator)**: An optimized version of the q-method, designed for real-time applications and quick computations.
- **OLAE (Optimal Linear Attitude Estimation)**: A linearized approach that simplifies calculations, particularly useful in scenarios with multiple observations.

Throughout the module, we’ll not only dive into how these methods work, but also discuss the pros and cons of each, especially in terms of computational efficiency and accuracy.

<ins>**Learning Objectives**</ins>

- **Determine attitude from a series of heading measurements**: Learn how to combine various directional observations to calculate a reliable orientation in 3D space.
- **Describe various classical and modern algorithms used in attitude determination**: Gain an understanding of the main methods used in both theoretical and practical applications.
- **Derive the fundamental attitude coordinate properties of rigid bodies**: Develop a mathematical grasp of the key properties that govern the rotational dynamics of rigid bodies in space.

---

In [1]:
# Import Relevant Libraries
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

import sys
sys.path.insert(0, r"../")
from AttitudeKinematicsLib import *

# 4.1) Attitude Determination Problem Statement

Attitude determination involves finding the orientation of an object (such as a spacecraft) in space, using instantaneous measurements of specific reference directions. This topic focuses on determining attitude without relying on dynamic filtering or rate measurements (e.g., gyroscopes), which are covered in more advanced topics like Kalman filters and require estimation theory. Here, the aim is to determine the orientation based solely on direct observations.


**<ins>Understanding Attitude Determination</ins>**

- **Objective**: The goal of attitude determination is to compute the orientation (attitude) of a spacecraft by aligning measured body-frame vectors with their corresponding known inertial-frame references. This orientation is typically represented as a Direction Cosine Matrix (DCM), $[\mathbf{B}\mathbf{N}]$, which maps vectors in the inertial frame ($\mathcal{^N}\hat{v}$) to the body frame ($\mathbf{^B}\hat{v}$).

- **Challenge**: Each sensor measurement provides partial information about the spacecraft’s 3D attitude. To resolve the full attitude, multiple independent observations are required. The problem involves relating noisy body-frame sensor measurements to accurate, well-known inertial-frame vectors.

---

**<ins>Inertial and Body Frame Vectors</ins>**

- **Inertial Frame Vectors ($\mathcal{^N}\hat{v}$)**:
  - These are **known reference directions** in an inertial coordinate system, such as the Sun’s direction, Earth’s magnetic field, or positions of stars.
  - They are computed using:
    1. **Position in Orbit**: Derived from GPS or ground-based tracking systems.
    2. **Time in Orbit**: Provided by onboard clocks or ground communication.
  - Using position and time, precise models (e.g., solar ephemeris, magnetic field models like IGRF) yield these vectors in the inertial frame.

<br>

- **Body Frame Vectors ($\mathbf{^B}\hat{v}$)**:
  - These are **measured directions** relative to the spacecraft’s body, obtained via onboard sensors:
    - **Sun Sensors**: Measure the Sun’s direction.
    - **Magnetometers**: Detect the Earth’s magnetic field.
    - **Star Trackers**: Determine the positions of stars.
  - Each sensor provides a body-frame vector pointing to a known reference object.

---

**<ins>Relating Inertial and Body Vectors</ins>**

- The relationship between measured body-frame vectors and their known inertial counterparts is described by the DCM:
  $$
  \mathbf{^B}\hat{v}_k = [\mathbf{B}\mathbf{N}] \mathcal{^N}\hat{v}_k, \quad k = 1, \dots, N
  $$
  where:
  - $\mathcal{^N}\hat{v}_k$: Known inertial-frame vector, derived from position, time, and reference models.
  - $\mathbf{^B}\hat{v}_k$: Measured body-frame vector, obtained from sensors.
    
- Conversely, the inverse DCM maps vectors from the body frame back to the inertial frame:
  $$
  \mathcal{^N}\hat{v}_k = [\mathbf{B}\mathbf{N}]^T \mathbf{^B}\hat{v}_k
  $$

---

**<ins>Key Insight</ins>**

- **Why Are Vectors Different?**:
  - The difference between body and inertial vectors reflects the spacecraft’s orientation in space.
  - If the body and inertial vectors match, it means the spacecraft's body and inertial frames coincide, i.e., no rotation.

<br>

- **Orientation Representation**:
  - The DCM, $[\mathbf{B}\mathbf{N}]$, represents how the body is oriented with respect to the inertial frame.
  - In practice, the spacecraft estimates $[\bar{\mathbf{B}}\mathbf{N}]$, an approximation of $[\mathbf{B}\mathbf{N}]$, based on body-frame sensor measurements and inertial-frame references.
  - This estimate allows us to determine how the body is oriented relative to the inertial frame, enabling precise navigation and control.

# 4.2) TRIAD Method

The Vector Triad Method is a deterministic approach to estimate a spacecraft's attitude at a given instant using two vector observations. This method introduces an intermediate frame, or "triad" frame, to simplify the problem. By aligning a third frame with both the inertial frame (e.g., stars or magnetic field) and the body frame (sensor measurements), the method allows a straightforward matrix solution to attitude estimation. Here’s an overview of the method and its key steps.

**<ins>Why Use the Vector Triad Method?</ins>**

- **Simple and Efficient**: The triad method is computationally inexpensive, making it suitable for missions with limited processing power.
- **Reliable for Limited Measurements**: When only two measurements are available (e.g., sun and magnetic field directions), this method provides a quick way to estimate attitude.
- **Optimal Choice of Measurement**: For the triad method, the measurement with the highest accuracy (usually the sun heading) is assigned as the primary reference axis. Less accurate measurements, such as the magnetic field, are used as secondary references.
le.

**<ins>Steps in the Vector Triad Method</ins>**

1. **Define Two Frames**:
   - **Inertial Frame**: Denoted as the $N$ frame, representing known directions in space, such as the position of the sun or Earth's magnetic field. These directions are assumed to be known because we generally know the satellite or spacecraft's position in its orbit relative to the Earth.

   - **Body Frame**: Denoted as the $B$ frame, representing measurements in the body’s coordinate system, such as sensor readings of the sun and magnetic field. Both the sun sensor and the magnetometer (MTM) provide vectors known in both the body frame and the inertial frame.

   - **Triad Frame**: An intermediate frame denoted as $T$, introduced to mitigate the effects of sensor noise. Determining the body frame attitude directly from noisy sensor data can lead to inaccuracies. Instead, the triad frame serves as a bridge between the inertial and body frames, stabilizing the attitude estimation process.

<br>

2. **Align the Primary Axis**:
   - Select the most accurate measurement (e.g., the sun heading) and align it with the first axis ($\mathbf{t}_1$) of the triad frame. The sun sensor typically provides a more precise measurement than the magnetic field sensor due to the variability of the Earth's magnetic field.

   - If the sun vector in the body frame is $\mathbf{s}_b$ and the sun vector in the inertial frame is $\mathbf{s}_n$, then:
     $$ \mathbf{t}_1 = \mathbf{s} $$

<br>

3. **Calculate the Orthogonal Vector for the Second Axis**:
   - Use the secondary measurement (e.g., magnetic field) to define a second axis orthogonal to the first axis.

   - For example, if $\mathbf{m}_b$ is the magnetic field in the body frame and $\mathbf{m}_n$ is the magnetic field in the inertial frame, calculate $\mathbf{t}_2$ as:
     $$ \mathbf{t}_2 = \frac{\mathbf{s} \times \mathbf{m}}{|\mathbf{s} \times \mathbf{m}|} $$

   - $\mathbf{t}_2$ is normalized to make it a unit vector, preseriving the directionality of the vector only.

<br>

4. **Define the Third Axis**:
   - Complete the triad by defining the third axis as the cross product of the first two:
     $$ \mathbf{t}_3 = \mathbf{t}_1 \times \mathbf{t}_2 $$

<br>

5. **Construct Transformation Matrices**:
   - The **T frame direction axes** can be computed using both **B** (body frame) and **N** (inertial frame) components:

     **Body Frame Triad Vectors**
     $$
     \mathbf{^B}\hat{t}_1 = \mathbf{^B}\hat{s}
     $$
     $$
     \mathbf{^B}\hat{t}_2 = \frac{\mathbf{^B}\hat{s} \times \mathbf{^B}\hat{m}}{|\mathbf{^B}\hat{s} \times \mathbf{^B}\hat{m}|}
     $$
     $$
     \mathbf{^B}\hat{t}_3 = \mathbf{^B}\hat{t}_1 \times \mathbf{^B}\hat{t}_2
     $$

     **Inertial Frame Triad Vectors**
     $$
     \mathcal{^N}\hat{t}_1 = \mathcal{^N}\hat{s}
     $$
     $$
     \mathcal{^N}\hat{t}_2 = \frac{\mathcal{^N}\hat{s} \times \mathcal{^N}\hat{m}}{|\mathcal{^N}\hat{s} \times \mathcal{^N}\hat{m}|}
     $$
     $$
     \mathcal{^N}\hat{t}_3 = \mathcal{^N}\hat{t}_1 \times \mathcal{^N}\hat{t}_2
     $$

   - In the absence of measurement errors, both sets of **Triad frame representations** should be identical.

   - Construct the **Body to Triad Matrix** ($[\bar{\mathbf{B}}\mathbf{T}]$) and **Inertial to Triad Matrix** ($[\mathcal{N}\mathbf{T}]$):
     $$
     [\bar{\mathbf{B}}\mathbf{T}] = \begin{bmatrix} \mathbf{^B}\hat{t}_1 & \mathbf{^B}\hat{t}_2 & \mathbf{^B}\hat{t}_3 \end{bmatrix}
     $$
     $$
     [\mathcal{N}\mathbf{T}] = \begin{bmatrix} \mathcal{^N}\hat{t}_1 & \mathcal{^N}\hat{t}_2 & \mathcal{^N}\hat{t}_3 \end{bmatrix}
     $$

<br>

6. **Calculate the Attitude Matrix**:
   - The final step is to compute the attitude matrix, $[\bar{\mathbf{B}}\mathbf{N}]$, that transforms from the inertial frame to the body frame:
     $$
     \bar{\mathbf{B}}\mathbf{N} = [\bar{\mathbf{B}}\mathbf{T}] \cdot [\mathcal{N}\mathbf{T}]^T
     $$
   - This product provides the full attitude matrix, which can then be used to extract desired attitude parameters (Euler angles, MRPs, etc.).

<br>

**Estimation Error Assessment**:
   - If the **actual orientation** ($[\mathbf{B}\mathbf{N}]$) is known, the estimation error can be computed as:
     $$
     [\bar{\mathbf{B}}\mathbf{B}] = [\bar{\mathbf{B}}\mathbf{N}] \cdot ([\mathbf{B}\mathbf{N}])^T
     $$
   - Here, $[\bar{\mathbf{B}}\mathbf{B}]$ represents the error matrix. If the estimation is perfect, $[\bar{\mathbf{B}}\mathbf{B}]$ will be the identity matrix.

   - To quantify the error, you can use **axis-angle formalism** to extract the rotation axis and angle from $[\bar{\mathbf{B}}\mathbf{B}]$. This will give a measure of how far the estimated body frame is from the true body frame in terms of rotation angle (radians or degrees).

   - This process allows for a precise evaluation of the accuracy of the attitude estimation.

**<ins>Important Considerations</ins>**

- **Measurement Accuracy**: The triad method assumes that the first measurement vector (e.g., sun heading) is more accurate. Aligning this vector with the primary axis ($\mathbf{t}_1$) uses its full information, while the secondary measurement provides additional orientation constraints.
- **Non-Collinearity of Measurements**: The two measurement vectors must not be collinear, as this would fail to constrain the third axis. For example, if the sun vector and magnetic field vector are aligned, the system would lose orientation around that axis.

**<ins>Practical Usage and Limitations</ins>**

- **Efficiency**: This method is fast and computationally light, making it suitable for real-time systems or constrained spacecraft systems.
- **Usage on Spacecraft**: The triad method has been implemented on various spacecraft where simplicity is prioritized, particularly for missions without advanced filtering methods.
- **Accuracy and Redundancy**: While effective, the method does not use all available data when only two measurements are used. As missions typically involve more sensors, more advanced algorithms like QUEST or Kalman filters can be employed to improve attitude estimation accuracy and handle dynamic environments.

**<ins>Summary</ins>**

The vector triad method is a foundational approach in attitude determination, especially valuable for its simplicity and direct use of two sensor measurements. It is a fundamental tool in aerospace engineering for quick and reliable orientation estimation when minimal data is available.

In [2]:
def triad_estimation(r1_b, r1_i, r2_b, r2_i):
    """
    Implements the TRIAD algorithm to compute the rotation matrix from the inertial frame to the body frame.

    Args:
        r1_b (array-like): First reference vector measured in the body frame (3-element array).
        r1_i (array-like): First reference vector in the inertial frame (3-element array).
        r2_b (array-like): Second reference vector measured in the body frame (3-element array).
        r2_i (array-like): Second reference vector in the inertial frame (3-element array).

    Returns:
        numpy.ndarray: Rotation matrix from the inertial frame to the body frame (3x3 matrix).
    """
    # Validate input vectors
    validate_vec3(r1_b)
    validate_vec3(r1_i)
    validate_vec3(r2_b)
    validate_vec3(r2_i)
    
    # Convert inputs to NumPy arrays and normalize the vectors
    r1_b = np.asarray(r1_b, dtype=float)
    r1_i = np.asarray(r1_i, dtype=float)
    r2_b = np.asarray(r2_b, dtype=float)
    r2_i = np.asarray(r2_i, dtype=float)

    r1_b /= np.linalg.norm(r1_b)
    r1_i /= np.linalg.norm(r1_i)
    r2_b /= np.linalg.norm(r2_b)
    r2_i /= np.linalg.norm(r2_i)

    # Construct the body-frame TRIAD vectors (estimated)
    t1_b = r1_b
    t2_b = np.cross(r1_b, r2_b)
    t2_b /= np.linalg.norm(t2_b)
    t3_b = np.cross(t1_b, t2_b)

    # Construct the inertial-frame TRIAD vectors
    t1_i = r1_i
    t2_i = np.cross(r1_i, r2_i)
    t2_i /= np.linalg.norm(t2_i)
    t3_i = np.cross(t1_i, t2_i)

    # Assemble the TRIAD matrices
    B_bar_T = np.column_stack((t1_b, t2_b, t3_b))  # Estimated body frame TRIAD matrix
    N_T = np.column_stack((t1_i, t2_i, t3_i))      # Inertial frame TRIAD matrix

    # Compute the rotation matrix from inertial frame to body frame
    C_BN = np.matmul(B_bar_T, N_T.T)

    return C_BN

# Example 3.15 from Textbook
v1_b = [0.8190, -0.5282, 0.2242]
v1_i = [1, 0, 0]
v2_b = [-0.3138, -0.1584, 0.9362]
v2_i = [0, 0, 1]
B_bar_N = triad_estimation(v1_b, v1_i, v2_b, v2_i)
print(B_bar_N)

[[ 0.81899104  0.45928237 -0.34396712]
 [-0.52819422  0.83763943 -0.13917991]
 [ 0.22419755  0.29566855  0.92860948]]


In [3]:
# Concept Check 2 - TRIAD Method, Q1
v1_b = [0.8273, 0.5541, -0.0920]
v1_i = [-0.1517, -0.9669, 0.2050]
v2_b = [-0.8285, 0.5522, -0.0955]
v2_i = [-0.8393, 0.4494, -0.3044]
B_bar_N = triad_estimation(v1_b, v1_i, v2_b, v2_i)
print(B_bar_N)

[[ 0.41555875 -0.85509088  0.31004921]
 [-0.83393237 -0.49427603 -0.24545471]
 [ 0.36313597 -0.15655922 -0.91848869]]


In [4]:
# Concept Check 2 - TRIAD Method, Q2

# Estimated attitude matrix (B_bar_N)
B_bar_N = np.array([
    [0.969846,  0.171010,  0.173648],
    [-0.200706, 0.964610,  0.171010],
    [-0.138258, -0.200706, 0.969846]
])

# True attitude matrix (BN)
BN = np.array([
    [0.963592,  0.187303,  0.190809],
    [-0.223042, 0.956645,  0.187303],
    [-0.147454, -0.223042, 0.963592]
])


B_bar_B = np.matmul(B_bar_N, BN.T)
axis, phi = DCM_to_PRV(B_bar_B)
print(phi)

1.8349476067250545


# 4.3) Wahba's Problem Defintion

**<ins>Wahba's Problem</ins>**

- **Objective**: To determine the optimal orientation (attitude) of a spacecraft by minimizing the error between measured body-frame vectors and their known inertial-frame counterparts. The orientation is represented as the Direction Cosine Matrix (DCM), $[\bar{\mathbf{B}}\mathbf{N}]$, which relates vectors in the inertial frame ($\mathcal{^N}\hat{v}$) to the body frame ($\mathbf{^B}\hat{v}$).

- **Challenge**: Sensor measurements are noisy and provide only partial information about the spacecraft's 3D attitude. The goal is to compute $[\bar{\mathbf{B}}\mathbf{N}]$ that minimizes the mismatch between measured and known vectors while accounting for sensor noise and variability.

---

**<ins>Mathematical Formulation</ins>**

1. **Mapping from Inertial to Body Frame**:  
   $$
   \mathbf{^B}\hat{v}_k = [\bar{\mathbf{B}}\mathbf{N}] \mathcal{^N}\hat{v}_k, \quad k = 1, \dots, N
   $$
   where:  
   - $\mathcal{^N}\hat{v}_k$: A known reference vector in the **inertial frame**, derived from models or ephemeris data (e.g., Sun direction, Earth's magnetic field).  
   - $\mathbf{^B}\hat{v}_k$: The corresponding vector **measured in the body frame** by onboard sensors (e.g., sun sensors, magnetometers).  
   - $N$: The number of vector pairs (observations) available.

<br>

2. **Error Minimization**:
   Wahba proposed a least squares approach to solve the attitude determination problem. The goal is to find $[\bar{\mathbf{B}}\mathbf{N}]$ that minimizes the cost function:
   $$
   J([\bar{\mathbf{B}}\mathbf{N}]) = \frac{1}{2} \sum_{k=1}^N w_k \| \mathbf{^B}\hat{v}_k - [\bar{\mathbf{B}}\mathbf{N}] \mathcal{^N}\hat{v}_k \|^2
   $$
   where:
   - $w_k$: Weight assigned to each observation, reflecting its accuracy or reliability.
   - $\|\cdot\|$: Euclidean norm.

   The factor of $\frac{1}{2}$ simplifies derivative calculations, canceling out a factor of 2 during optimization.

<br>

3. **Perfect Measurements**:
   If all measurements are noise-free, the cost function becomes zero:
   $$
   J = 0
   $$

---

**<ins>Why Multiple Measurements?</ins>**

- Each measurement vector provides only two independent pieces of information (e.g., azimuth and elevation). Therefore:
  - A single vector cannot resolve all three degrees of freedom for 3D orientation.
  - At least two independent reference vectors are needed to compute the attitude matrix uniquely.
  - When $N > 2$, the problem becomes over-specified, and the least squares approach improves robustness against noise.

---

**<ins>Analogy to Least Squares Fitting</ins>**

- The cost function $J([\bar{\mathbf{B}}\mathbf{N}])$ resembles the least squares regression problem:
  - In regression, we minimize the squared error between predicted and observed data.
  - In Wahba's problem, we minimize the squared angular error between measured body-frame vectors and their corresponding inertial-frame references.

Thus, the DCM $[\bar{\mathbf{B}}\mathbf{N}]$ is the "best-fit rotation matrix" that aligns the reference vectors with the observations.

---

**<ins>Importance of Weights</ins>**

- Sensors vary in accuracy:
  - **Sun Sensors**: Typically more accurate and reliable.
  - **Magnetometers (MTM)**: More prone to noise due to environmental variations.
- The weights $w_k$ reflect the reliability of each sensor. Higher weights are assigned to more reliable measurements, ensuring they contribute more to the solution.
- Scaling all weights $w_k$ uniformly does not affect the result because the optimization is relative.

---

**<ins>Estimation Accuracy Check</ins>**

1. **Compute Estimation Error**:
   After estimating $[\bar{\mathbf{B}}\mathbf{N}]$, compare it to the true DCM $[\mathbf{B}\mathbf{N}]$ (if known):
   $$
   [\mathbf{B}\bar{\mathbf{B}}] = [\bar{\mathbf{B}}\mathbf{N}] \cdot [\mathbf{B}\mathbf{N}]^T
   $$
   - $[\mathbf{B}\bar{\mathbf{B}}]$: The "error DCM" quantifies the deviation between estimated and true attitudes.

<br>

2. **Error in Terms of Axis-Angle**:
   - If the estimate is perfect:
     $$
     [\mathbf{B}\bar{\mathbf{B}}] = \mathbf{I}
     $$
   - Otherwise, compute the axis and angle of rotation using axis-angle formalism. The rotation angle represents the magnitude of the estimation error.

---

**<ins>Summary</ins>**

Wahba's problem frames attitude determination as a least squares optimization challenge. The core idea is to compute the norm of the difference between the measured body-frame vectors and the mapped inertial-frame vectors using the DCM $[\bar{\mathbf{B}}\mathbf{N}]$. 

- If the norm is **zero**, it indicates perfect alignment: the DCM $[\bar{\mathbf{B}}\mathbf{N}]$ has mapped the inertial vectors perfectly onto their body-frame counterparts, and $J = 0$. 
- If the norm is **non-zero**, it reflects an error in mapping caused by sensor noise or inaccuracies, resulting in $J > 0$. 

This interpretation makes Wahba's problem a direct extension of least squares fitting to attitude determination, where the objective is to minimize this norm and find the best-fit DCM.

# 4.4) Devenport's q-Method

Devenport's q-Method is a powerful technique for determining the attitude of a spacecraft using quaternions. It reformulates Wahba's problem into an eigenvalue problem, efficiently finding the optimal quaternion that describes the attitude.

**<ins>Objective</ins>**

- The goal is to find the quaternion $\beta = [\beta_0, \beta_1, \beta_2, \beta_3]^T$, where:
  - $\beta_0$ is the scalar part.
  - $\beta_1, \beta_2, \beta_3$ form the vector part.
  
- The quaternion $\beta$ is a **unit vector** satisfying $\|\beta\|^2 = 1$, ensuring it represents a valid rotation.

- This method minimizes the Wahba cost function:
  $$
  J([\bar{\mathbf{B}}\mathbf{N}]) = \frac{1}{2} \sum_{k=1}^N w_k \| \mathbf{^B}\hat{v}_k - [\bar{\mathbf{B}}\mathbf{N}] \mathcal{^N}\hat{v}_k \|^2
  $$
  and equivalently maximizes the **gain function**:
  $$
  g(\beta) = \beta^T [K] \beta
  $$
  where $[K]$ is the **gain matrix**, which encodes the relationship between the observed and known reference vectors.

- Note that $\mathbf{^B}\hat{v}_k$ and $\mathcal{^N}\hat{v}_k$ are vectors in $\mathcal{i, j, k}$
---

**<ins>Re-Writing Wahba Cost Function & Identifying the Gain Function, g</ins>**

1. **Rewriting the Wahba Cost Function**:  
   - The Wahba cost function measures the mismatch between the body-frame vectors $\mathbf{^B}\hat{v}_k$ and the inertial-frame vectors $\mathcal{^N}\hat{v}_k$ mapped via the Direction Cosine Matrix (DCM) $[\bar{\mathbf{B}}\mathbf{N}]$:
     $$
     J([\bar{\mathbf{B}}\mathbf{N}]) = \frac{1}{2} \sum_{k=1}^N w_k \| \mathbf{^B}\hat{v}_k - [\bar{\mathbf{B}}\mathbf{N}] \mathcal{^N}\hat{v}_k \|^2
     $$

   - The cost function can be re-written as:
     $$
     J = \frac{1}{2} \sum_{k=1}^N w_k \left( \mathbf{^B}\hat{v}_k - [\bar{\mathbf{B}}\mathbf{N}] \mathcal{^N}\hat{v}_k \right)^T \left( \mathbf{^B}\hat{v}_k - [\bar{\mathbf{B}}\mathbf{N}] \mathcal{^N}\hat{v}_k \right)
     $$

   - Further expansion of the inner product:
     $$
     J = \frac{1}{2} \sum_{k=1}^N w_k \left( (\mathbf{^B}\hat{v}_k)^T (\mathbf{^B}\hat{v}_k) + (\mathcal{^N}\hat{v}_k)^T [\bar{\mathbf{B}}\mathbf{N}]^T [\bar{\mathbf{B}}\mathbf{N}] (\mathcal{^N}\hat{v}_k) - 2 (\mathbf{^B}\hat{v}_k)^T [\bar{\mathbf{B}}\mathbf{N}] (\mathcal{^N}\hat{v}_k) \right)
     $$

   - Since $\mathbf{^B}\hat{v}_k$ and $\mathcal{^N}\hat{v}_k$ are unit vectors:
     $$
     (\mathbf{^B}\hat{v}_k)^T (\mathbf{^B}\hat{v}_k) = 1, \quad (\mathcal{^N}\hat{v}_k)^T (\mathcal{^N}\hat{v}_k) = 1
     $$
     and,
     $$
     [\bar{\mathbf{B}}\mathbf{N}]^T [\bar{\mathbf{B}}\mathbf{N}] = I
     $$
     the expression simplifies to:
     $$
     J([\bar{\mathbf{B}}\mathbf{N}]) = \frac{1}{2} \sum_{k=1}^N w_k \left( 2 - 2 \mathbf{^B}\hat{v}_k^T [\bar{\mathbf{B}}\mathbf{N}] \mathcal{^N}\hat{v}_k \right)
     $$

   - Factoring out $2$:
     $$
     J([\bar{\mathbf{B}}\mathbf{N}]) = \sum_{k=1}^N w_k \left( 1 - \mathbf{^B}\hat{v}_k^T [\bar{\mathbf{B}}\mathbf{N}] \mathcal{^N}\hat{v}_k \right)
     $$


2. **The Gain Function**:
   - Since minimizing $J$ is equivalent to maximizing:
     $$
     g = \sum_{k=1}^N w_k \mathbf{^B}\hat{v}_k^T [\bar{\mathbf{B}}\mathbf{N}] \mathcal{^N}\hat{v}_k
     $$
   - The gain function $g$ measures the alignment between the body-frame and inertial-frame vectors.

---

**<ins>Gain Function in Terms of Quaternions</ins>**

1. **Expressing the DCM $[\bar{\mathbf{B}}\mathbf{N}]$ in Quaternions**:
   $$
   [\bar{\mathbf{B}}\mathbf{N}] = (\beta_0^2 - \epsilon^T \epsilon)[I_{3 \times 3}] + 2\epsilon\epsilon^T - 2\beta_0[\tilde{\epsilon}]
   $$
   where:
   - $\epsilon = [\beta_1, \beta_2, \beta_3]^T$ is the vector part of the quaternion.
   - $[\tilde{\epsilon}]$ is the skew-symmetric matrix of $\epsilon$.

<br>

2. **Simplifying $g$**:
   - Substituting the quaternion representation of $[\bar{\mathbf{B}}\mathbf{N}]$ into $g$ gives:
     $$
     g(\beta) = \beta^T [K] \beta
     $$
   - $[K]$ is the **gain matrix**

---

**<ins>Constructing the Gain Matrix $[K]$</ins>**

1. Compute the **Attitude Profile Matrix** $[B]$:  
   - The matrix $[B]$ is defined as:
     $$
     [B] = \sum_{k=1}^N w_k \mathbf{^B}\hat{v}_k (\mathcal{^N}\hat{v}_k)^T
     $$
   - **What is the Attitude Profile Matrix?**  
     $[B]$ is called the **Attitude Profile Matrix** because it encapsulates the relationship between the measured body-frame vectors $\mathbf{^B}\hat{v}_k$ and the known inertial-frame vectors $\mathcal{^N}\hat{v}_k$. It essentially "profiles" or summarizes the rotational alignment between these vectors, weighted by their relative accuracies $w_k$.  
     - The outer product $\mathbf{^B}\hat{v}_k (\mathcal{^N}\hat{v}_k)^T$ aligns the body-frame and inertial-frame vectors for each observation $k$.  
     - Summing over all observations combines these alignments into a single matrix, which serves as the foundation for computing the optimal quaternion or Direction Cosine Matrix (DCM).

<br>

2. Compute $\sigma$, $[Z]$, and $[S]$ using $[B]$:  
   - $\sigma = \text{tr}([B])$: The trace of $[B]$, representing the sum of its diagonal elements.  
   - $[Z] = [B_{23} - B_{32}, B_{31} - B_{13}, B_{12} - B_{21}]^T$: A 3D vector derived from the off-diagonal elements of $[B]$.  
   - $[S] = [B] + [B]^T$: The symmetric part of $[B]$.

<br>

3. Assemble the **Gain Matrix** $[K]$:  
   $$
   [K] = \begin{bmatrix} 
   \sigma & Z^T \\ 
   Z & [S] - \sigma [I_{3 \times 3}] 
   \end{bmatrix}
   $$


---

**<ins>Constrained Optimization</ins>**

1. **Unit Quaternion Constraint**:
   - Quaternions satisfy the holonomic constraint:
     $$
     \|\beta\|^2 = 1 \quad \Rightarrow \quad \beta^T\beta = 1
     $$

2. **Optimization Using Lagrange Multipliers**:
   - A **Lagrange multiplier** is a mathematical tool used in constrained optimization to incorporate constraints into the objective function. By introducing the multiplier $\lambda$, we reformulate the optimization problem so that the constraint $\beta^T\beta = 1$ is explicitly enforced while maximizing the gain function $g(\beta)$. 
   
   - In the general case, the Lagrange function is given by:
     $$
     \mathcal{L}(x, \lambda) = f(x) - \lambda h(x)
     $$
     where:
     - $f(x)$ is the objective function to be maximized or minimized.
     - $h(x)$ is the constraint that must be satisfied (e.g., $h(x) = 0$).
     - $\lambda$ is the Lagrange multiplier.

   - Here, the gain function $g(\beta)$ is maximized under the unit quaternion constraint $\|\beta\|^2 = 1$, which can be written as:
     $$
     g'(\beta) = g(\beta) - \lambda(\|\beta\|^2 - 1)
     $$
     Substituting for $g(\beta)$, we get:
     $$
     g'(\beta) = \beta^T[K]\beta - \lambda(\beta^T\beta - 1)
     $$

   - Differentiating $g'(\beta)$ with respect to $\beta$ to find its critical points:
     $$
     \frac{\partial g'}{\partial \beta} = 2[K]\beta - 2\lambda\beta = 0
     $$

   - Rearranging gives:
     $$
     [K]\beta = \lambda\beta
     $$

   - This result shows that $\beta$ must be an eigenvector of the matrix $[K]$, and $\lambda$ is the corresponding eigenvalue.

3. **Eigenvalue Problem**:
   - The solution is the eigenvector $\beta$ corresponding to the largest eigenvalue $\lambda$ of $[K]$. This eigenvector represents the quaternion that maximizes the gain function $g(\beta)$ while satisfying the unit norm constraint.


---

**<ins>Solution Procedure</ins>**

1. Compute the $[B]$ matrix from measurements.

2. Construct the $[K]$ matrix.

3. Solve the eigenvalue problem $[K]\beta = \lambda\beta$.

4. Select the eigenvector associated with the largest eigenvalue $\lambda$. This is the optimal quaternion $\beta$.

---

**<ins>Summary</ins>**

Davenport's Q-Method provides a robust framework for static attitude determination by solving Wahba's problem through a series of well-defined mathematical steps:

1. **Wahba's Problem**: The method starts with the Wahba cost function, which quantifies the mismatch between measured body-frame vectors and corresponding inertial-frame vectors after applying a candidate rotation. The goal is to minimize this cost, which corresponds to finding the optimal alignment.

2. **Linear Algebra Insights**: Using properties of linear algebra, the squared error terms in Wahba's problem are rewritten as inner products. This reformulation simplifies the problem and allows for the identification of the **gain function** $g(\beta)$, which is to be maximized.

3. **Gain Function in Quaternion Form**: The gain function $g(\beta) = \beta^T K \beta$ is expressed in terms of quaternions, enabling the use of quaternion algebra to represent the rotation. The focus then shifts to constructing the **gain matrix** $K$, which encodes the relationship between the observed vectors and their reference counterparts.

4. **Constructing the Gain Matrix $K$**: The matrix $K$ is built from the **attitude profile matrix** $B$, which is computed as a weighted sum of outer products of the body and inertial vectors. The symmetric properties of $B$ and its trace ($\sigma$) are used to populate the elements of $K$.

5. **Optimization with Constraints**: To enforce the unit quaternion constraint ($\|\beta\|^2 = 1$), the method introduces a Lagrange multiplier. This transforms the problem into an eigenvalue-eigenvector formulation, where the optimal quaternion corresponds to the eigenvector associated with the largest eigenvalue of $K$.

6. **Solution via Eigenvalue Problem**: Solving the eigenvalue problem yields the optimal quaternion $\beta$. The quaternion is normalized to ensure it satisfies the unit constraint.

7. **Constructing the Optimal $[\bar{\mathbf{B}}\mathbf{N}]$ Matrix**: The optimal quaternion is used to estimate the Direction Cosine Matrix (DCM) $[\bar{\mathbf{B}}\mathbf{N}]$, which represents the best alignment between the body and inertial frames.

Through this sequence of steps, Davenport's Q-Method elegantly combines linear algebra, quaternion representation, and optimization to solve for the attitude efficiently. It is computationally intensive at step 6, so it might not be efficient for real-time uses, but it lays a solid foundation for advanced attitude determination techniques in aerospace and related fields.

In [5]:
def davenport_q_method(B_v_k, N_v_k, weights=None):
    """
    Implements Davenport's Q-Method to estimate the rotation matrix (DCM) from inertial frame to body frame.

    Args:
        B_v_k (numpy.ndarray): Array of shape (N, 3) containing N body-frame vectors \( \mathbf{^B}\hat{v}_k \).
                               Each row corresponds to the k-th body-frame vector.
        
        N_v_k (numpy.ndarray): Array of shape (N, 3) containing N inertial-frame vectors \( \mathcal{^N}\hat{v}_k \).
                               Each row corresponds to the k-th inertial-frame vector.
        
        weights (numpy.ndarray, optional): Array of shape (N,) containing weights \( w_k \) for each vector pair.
                                           If None, equal weights are assumed.

    Returns:
        numpy.ndarray: Estimated rotation matrix (3x3) \( [\bar{\mathbf{B}}\mathbf{N}] \) from inertial frame to body frame.

    Notes:
        - \( N \) is the number of vector observations.
        - Each vector is a 3-dimensional vector (size 3).
        - The vectors should be provided as NumPy arrays and will be normalized to unit length within the function.
        - The function computes the optimal rotation matrix that aligns the inertial-frame vectors \( \mathcal{^N}\hat{v}_k \)
          with the body-frame vectors \( \mathbf{^B}\hat{v}_k \) by minimizing Wahba's cost function.

    Example:
        >>> B_v_k = np.array([[0.8273, 0.5541, -0.0920],
                              [-0.8285, 0.5522, -0.0955],
                              [0.5774, 0.5774, 0.5774]])
        >>> N_v_k = np.array([[-0.1517, -0.9669, 0.2050],
                              [-0.8393, 0.4494, -0.3044],
                              [0.5774, 0.5774, 0.5774]])
        >>> weights = np.array([1.0, 1.0, 1.0])
        >>> C = davenport_q_method(B_v_k, N_v_k, weights)
        >>> print(C)
    """
    # Ensure inputs are lists or arrays
    if not isinstance(B_v_k, (list, np.ndarray)):
        raise TypeError("B_v_k must be a list or NumPy array of vectors.")
    if not isinstance(N_v_k, (list, np.ndarray)):
        raise TypeError("N_v_k must be a list or NumPy array of vectors.")
        
    # Validate input dimensions
    if B_v_k.shape != N_v_k.shape:
        raise ValueError("Input vector arrays must have the same shape. The number of vector observations must be the same in body and inertial frame.")

    # Number of vector observations
    N = B_v_k.shape[0]

    # Cast vectors to float 
    B_v_k = np.asarray(B_v_k, dtype=float)
    N_v_k = np.asarray(N_v_k, dtype=float)
    
    # Validate each vector in B_v_k and N_v_k
    for k in range(N):
        validate_vec3(B_v_k[k])
        validate_vec3(N_v_k[k])
    
    if weights is None:
        weights = np.ones(N)
    else:
        weights = np.asarray(weights, dtype=float)
        if weights.shape[0] != N:
            raise ValueError("Weights array must have the same length as the number of vector observations.")
    
    # Normalize the input vectors to unit length
    B_v_k /= np.linalg.norm(B_v_k, axis=1, keepdims=True)
    N_v_k /= np.linalg.norm(N_v_k, axis=1, keepdims=True)
    
    # Construct the attitude profile matrix B
    B = np.zeros((3, 3))
    for k in range(N):
        B += weights[k] * np.outer(B_v_k[k], N_v_k[k])
    
    # Compute the symmetric matrix S
    S = B + B.T

    # Compute the vector sigma
    sigma = np.array([B[1, 2] - B[2, 1],
                      B[2, 0] - B[0, 2],
                      B[0, 1] - B[1, 0]])
    
    # Compute the scalar zeta
    zeta = np.trace(B)
    
    # Construct the K matrix
    K = np.zeros((4, 4))
    K[0, 0] = zeta
    K[0, 1:] = sigma
    K[1:, 0] = sigma
    K[1:, 1:] = S - zeta * np.eye(3)
    
    # Solve for the eigenvector corresponding to the maximum eigenvalue
    eigenvalues, eigenvectors = np.linalg.eig(K)
    max_index = np.argmax(eigenvalues)
    q_optimal = eigenvectors[:, max_index]
    
    # Normalize the quaternion
    q_optimal /= np.linalg.norm(q_optimal)
    
    # Convert quaternion to rotation matrix
    C = EP_to_DCM(q_optimal)
    
    return C

In [6]:
# Concept Check 3, 4 - Devenport's q-method
v_b = np.array([
    [0.8273, 0.5541, -0.0920],
    [-0.8285, 0.5522, -0.0955]
])

v_i = np.array([
    [-0.1517, -0.9669, 0.2050],
    [-0.8393, 0.4494, -0.3044]
])

# Call the Davenport Q-Method function
rotation_matrix = davenport_q_method(v_b, v_i)

# Print the resulting Direction Cosine Matrix (DCM)
print("Estimated Direction Cosine Matrix (DCM):")
print(np.round(rotation_matrix, 6))

Estimated Direction Cosine Matrix (DCM):
[[ 0.415936 -0.854894  0.310087]
 [-0.833757 -0.494637 -0.245325]
 [ 0.363107 -0.156498 -0.918511]]


# 4.5) QUEST

**<ins>Objective</ins>**

- The QUEST (QUaternion ESTimator) method is a faster solution to the Wahba problem compared to Davenport's Q-Method. It avoids directly solving the computationally intensive eigenvalue-eigenvector problem.
  
- The goal is to find the quaternion $\beta = [\beta_0, \beta_1, \beta_2, \beta_3]^T$, where:
  - $\beta_0$ is the scalar part.
  - $\beta_1, \beta_2, \beta_3$ form the vector part.

- The quaternion $\beta$ is a **unit vector** satisfying $\|\beta\|^2 = 1$, ensuring it represents a valid rotation.

- Like Davenport's Q-Method, QUEST minimizes the Wahba cost function:
  $$
  J([\bar{\mathbf{B}}\mathbf{N}]) = \frac{1}{2} \sum_{k=1}^N w_k \| \mathbf{^B}\hat{v}_k - [\bar{\mathbf{B}}\mathbf{N}] \mathcal{^N}\hat{v}_k \|^2
  $$

---

**<ins>Rewriting Wahba's Cost Function and Introducing the Optimal Eigenvalue</ins>**

1. **Linking Cost and Gain Functions**:
   - The cost function $J$ can be rewritten using the gain function $g$:
     $$
     J = \sum_{k=1}^N w_k - g
     $$
   - From Davenport's q-method, for the optimal quaternion $\bar\beta$, the gain function $g$ equals the largest eigenvalue $\lambda_{\text{opt}}$ of the gain matrix $[K]$:
     $$
     g(\bar\beta) = \lambda_{\text{opt}}
     $$

2. **Simplified Cost Function**:
   - Substituting $g(\bar\beta) = \lambda_{\text{opt}}$ into the cost function:
     $$
     J = \sum_{k=1}^N w_k - \lambda_{\text{opt}}
     $$

3. **Expression for the Optimal Eigenvalue**:
   - Rearranging for $\lambda_{\text{opt}}$ gives:
     $$
     \lambda_{\text{opt}} = \sum_{k=1}^N w_k - J
     $$
   - If the measurement noise is small (as assumed in QUEST), $J$ is close to zero, so:
     $$
     \lambda_{\text{opt}} \approx \sum_{k=1}^N w_k
     $$

---

**<ins>Numerical Optimization via Root Solving</ins>**

1. **Avoiding the Eigenvalue Problem**:
   - Instead of solving the eigenvalue problem for $[K]$, QUEST approximates $\lambda_{\text{opt}}$ using an iterative **root-solving method** (e.g., Newton-Raphson).

2. **Characteristic Equation**:
   - The eigenvalues of $[K]$ are roots of the characteristic equation:
     $$
     f(s) = \text{det}([K] - s[I_{4 \times 4}]) = 0
     $$

3. **Newton-Raphson Iteration**:
   - To solve for the largest root $\lambda_{\text{opt}}$, Newton-Raphson iteratively refines the estimate:
     $$
     \lambda_{i+1} = \lambda_i - \frac{f(\lambda_i)}{f'(\lambda_i)}
     $$
   - This method efficiently converges to the largest eigenvalue starting from an initial guess $\lambda_0 = \sum_{k=1}^N w_k$.

---

**<ins>Rodrigues Parameters and Attitude Reconstruction</ins>**

1. **Classical Rodrigues Parameters (CRPs)**:
   - To simplify the quaternion representation, QUEST introduces the **Rodrigues parameter vector**:
     $$
     \mathbf{q} = \frac{\epsilon}{\beta_0} = \begin{bmatrix} \beta_1 / \beta_0 \\ \beta_2 / \beta_0 \\ \beta_3 / \beta_0 \end{bmatrix}
     $$
     where $\epsilon = [\beta_1, \beta_2, \beta_3]^T$ is the vector part of the quaternion.

2. **Linearized Eigenvector Problem**:
   - The eigenvector problem $[K] \beta = \lambda_{\text{opt}} \beta$ is rewritten as:
     $$
     \left([S] - \lambda_{\text{opt}} [I_{3 \times 3}] + \sigma\right) \mathbf{q} = [Z]
     $$
   - Here, $[S]$, $[Z]$, and $\sigma$ are derived from the attitude profile matrix $[B]$.

3. **Solving for CRPs**:
   - The CRPs are obtained by solving:
     $$
     \mathbf{q} = \left((\lambda_{\text{opt}} + \sigma)[I_{3 \times 3}] - [S]\right)^{-1} [Z]
     $$

4. **Reconstructing the Quaternion**:
   - From $\mathbf{q}$, the quaternion is reconstructed as:
     $$
     \beta = \frac{1}{\sqrt{1 + \mathbf{q}^T \mathbf{q}}} \begin{bmatrix} 1 \\ \mathbf{q} \end{bmatrix}
     $$

---

**<ins>Advantages of QUEST</ins>**

1. **Speed**:
   - Avoids direct eigenvalue decomposition, making it computationally faster, especially for real-time applications.
2. **Iterative Precision**:
   - Achieves high precision through iterative root solving (e.g., Newton-Raphson).
3. **Flexibility**:
   - Supports various weighting schemes and multiple vector observations.

---

**<ins>Summary</ins>**

The QUEST method builds on Davenport's Q-Method to solve Wahba's problem more efficiently:

1. **Wahba's Cost Function**: Minimizes $J$ while maximizing the gain function $g$, which corresponds to the largest eigenvalue of $[K]$.
2. **Avoiding Eigenvalue Decomposition**: Uses Newton-Raphson to iteratively solve for the largest eigenvalue $\lambda_{\text{opt}}$ instead of direct matrix diagonalization.
3. **Attitude Representation**: Employs Classical Rodrigues Parameters (CRPs) to simplify quaternion representation and reconstructs the optimal quaternion $\beta$.
4. **Practical Advantages**: Combines numerical speed with high accuracy, making it well-suited for spacecraft attitude determination in real-time applications.

QUEST exemplifies an elegant and practical refinement of Davenport's Q-Method, maintaining accuracy while significantly improving computational efficiency.


In [7]:
def quest_algorithm(B_v_k, N_v_k, weights=None, max_iter=1000, tol=1e-12):
    """
    Implements the QUEST algorithm to estimate the rotation matrix (DCM) from inertial frame to body frame.
    
    Args:
        B_v_k (numpy.ndarray): Array of shape (N, 3) containing N body-frame vectors \( \mathbf{^B}\hat{v}_k \).
                               Each row corresponds to the k-th body-frame vector.
    
        N_v_k (numpy.ndarray): Array of shape (N, 3) containing N inertial-frame vectors \( \mathcal{^N}\hat{v}_k \).
                               Each row corresponds to the k-th inertial-frame vector.
    
        weights (numpy.ndarray, optional): Array of shape (N,) containing weights \( w_k \) for each vector pair.
                                           If None, equal weights are assumed.
    
        max_iter (int, optional): Maximum number of iterations for the Newton-Raphson method. Default is 100.
    
        tol (float, optional): Tolerance for convergence of the eigenvalue. Default is 1e-12.
    
    Returns:
        numpy.ndarray: Estimated rotation matrix (3x3).
    """
    # Ensure inputs are NumPy arrays
    B_v_k = np.asarray(B_v_k, dtype=float)
    N_v_k = np.asarray(N_v_k, dtype=float)

    # Validate input dimensions
    if B_v_k.shape != N_v_k.shape:
        raise ValueError("Input vector arrays must have the same shape.")

    # Number of vector observations
    N = B_v_k.shape[0]

    # Assign equal weights if none provided
    if weights is None:
        weights = np.ones(N)
    else:
        weights = np.asarray(weights, dtype=float)
        if weights.shape[0] != N:
            raise ValueError("Weights array must have the same length as the number of vector observations.")

    # Normalize the input vectors to unit length
    B_v_k /= np.linalg.norm(B_v_k, axis=1, keepdims=True)
    N_v_k /= np.linalg.norm(N_v_k, axis=1, keepdims=True)

    # Construct the attitude profile matrix B
    B = np.zeros((3, 3))
    for k in range(N):
        B += weights[k] * np.outer(B_v_k[k], N_v_k[k])

    # Compute scalar quantities
    I = np.eye(3)
    S = B + B.T  # Symmetric part of B
    sigma = np.trace(B)
    Z = np.array([B[1, 2] - B[2, 1],
                  B[2, 0] - B[0, 2],
                  B[0, 1] - B[1, 0]])

    # Construct the K matrix
    K = np.zeros((4, 4))
    K[0, 0] = sigma
    K[0, 1:] = Z
    K[1:, 0] = Z
    K[1:, 1:] = S - sigma * np.eye(3)

    # Newton-Raphson for largest eigenvalue
    def f(s):
        """Characteristic equation f(s) = det(K - s * I)."""
        return np.linalg.det(K - s * np.eye(4))

    def f_prime(s):
        """Derivative of the characteristic equation f'(s)."""
        K_sI = K - s * np.eye(4)
        inv_K_sI = np.linalg.inv(K_sI)
        return -f(s) * np.trace(inv_K_sI)

    # Initial guess for lambda (the largest eigenvalue)
    lambda_new = np.sum(weights)

    # Perform Newton-Raphson iteration
    for _ in range(max_iter):
        f_val = f(lambda_new)
        f_prime_val = f_prime(lambda_new)

        # Avoid division by zero
        if f_prime_val == 0:
            raise ZeroDivisionError("Derivative is zero during Newton-Raphson iteration.")

        delta_lambda = -f_val / f_prime_val
        lambda_new += delta_lambda

        if np.abs(delta_lambda) < tol:
            break
    else:
        raise RuntimeError("Newton-Raphson did not converge within the maximum number of iterations.")

    # Compute Rodrigues parameter vector (q)
    A = (lambda_new + sigma) * np.eye(3) - S
    q = np.linalg.solve(A, Z)

    # Construct quaternion
    beta = np.zeros(4)
    beta[0] = 1 / np.sqrt(1 + np.dot(q, q))
    beta[1:] = beta[0] * q

    # Convert quaternion to DCM
    C = EP_to_DCM(beta)

    return C

In [8]:
# Concept Check 5 - QUEST
v_b = np.array([
    [0.8273, 0.5541, -0.0920],
    [-0.8285, 0.5522, -0.0955]
])

v_i = np.array([
    [-0.1517, -0.9669, 0.2050],
    [-0.8393, 0.4494, -0.3044]
])

# Call the QUEST function
DCM_estimated = quest_algorithm(v_b, v_i)

# Print the resulting Direction Cosine Matrix (DCM)
print("Estimated DCM:")
print(np.round(DCM_estimated, 6))

Estimated DCM:
[[ 0.415936 -0.854894  0.310087]
 [-0.833757 -0.494637 -0.245325]
 [ 0.363107 -0.156498 -0.918511]]


In [9]:
import time

# Define the test vectors
v_b = np.array([
    [0.8273, 0.5541, -0.0920],
    [-0.8285, 0.5522, -0.0955]
])

v_i = np.array([
    [-0.1517, -0.9669, 0.2050],
    [-0.8393, 0.4494, -0.3044]
])

# Measure time for Davenport's Q-Method
start_time_davenport = time.time()
DCM_davenport = davenport_q_method(v_b, v_i)
end_time_davenport = time.time()
time_davenport = end_time_davenport - start_time_davenport

# Print results for Davenport's Q-Method
print("Davenport's Q-Method:")
print("Estimated DCM:")
print(np.round(DCM_davenport, 6))
print(f"Time taken: {time_davenport:.12f} seconds\n")

# Measure time for QUEST algorithm
start_time_quest = time.time()
DCM_quest = quest_algorithm(v_b, v_i)
end_time_quest = time.time()
time_quest = end_time_quest - start_time_quest

# Print results for QUEST algorithm
print("QUEST Algorithm:")
print("Estimated DCM:")
print(np.round(DCM_quest, 6))
print(f"Time taken: {time_quest:.12f} seconds\n")


Davenport's Q-Method:
Estimated DCM:
[[ 0.415936 -0.854894  0.310087]
 [-0.833757 -0.494637 -0.245325]
 [ 0.363107 -0.156498 -0.918511]]
Time taken: 0.000999927521 seconds

QUEST Algorithm:
Estimated DCM:
[[ 0.415936 -0.854894  0.310087]
 [-0.833757 -0.494637 -0.245325]
 [ 0.363107 -0.156498 -0.918511]]
Time taken: 0.000000000000 seconds



# 4.6) OLAE

# 4.7) Kinematics Final Assignment